In [ ]:
import pandas as pd
from collections import defaultdict
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import json
import gzip
import pickle 

# Code for proprocessing of text 
### Used for tokenization and other preprocessing

In [ ]:
#for stopword removal
nltk.download('stopwords')
stop = set(stopwords.words('english') + list(string.punctuation))


# Function to tokenize and preprocess a document
def preprocess(text):
    tokens = set(word_tokenize(text.lower())) # get all tokens
    return [i for i in tokens if i not in stop] # get all tokens without stopwords

    # add lematization later

# Creation of ininverted index

In [ ]:
OUTPUT_PATH = "inverted_index.jsonl"

#create terms used in each document
#extend with count
corpus = 'trec-tot-2025-corpus.jsonl.gz'

print("Starting with the creation of the inverted index (6,407,814 entries)") # based on the number on the website
count = 0 # print progress

inverted_index = defaultdict(list)
with gzip.open(corpus, 'rt', encoding='utf-8') as f_in, \
    open('inverted_index.pkl', 'wb') as f_out:
    for line_number, line in enumerate(f_in, start=1):
        line = line.strip()
        if not line:
            continue  # skip empty lines
        try:
            data = json.loads(line)
            text = data.get("text", "")
            tokens = preprocess(text)
            set(tokens)                         #one for all later ad counts as well
            for token in tokens:
                inverted_index[token].append(line_number)
        except json.JSONDecodeError as e:
            print(f"[Line {line_number}] Invalid JSON: {e}")

        count += 1
        if count % 10000 == 0: # ginve information of progress and saving it to a file
            print(count)
            pickle.dump(dict(inverted_index), f_out) # saving it
            f_out.flush

print(f"Finished creation of the inverted index.")

# Opening the inverted index

In [24]:
with open('inverted_index.pkl', 'rb') as f:
    inverted_index = pickle.load(f)

print(inverted_index)

{'second': [1, 2, 3, 4, 5, 6, 7, 8, 10, 13, 15, 18, 20, 22, 23, 25, 27, 28, 29, 31, 32, 33, 35, 37, 38, 39, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 56, 57, 58, 60, 63, 66, 67, 73, 74, 75, 77, 78, 79, 81, 86, 87, 88, 91, 92, 93, 94, 96, 97, 99], 'politics': [1, 4, 6, 7, 14, 15, 18, 19, 20, 29, 32, 35, 37, 39, 61, 74, 78, 79, 80, 81, 86, 88, 91, 92, 93, 96], 'emphasis': [1, 3, 5, 6, 7, 14, 15, 20, 22, 28, 32, 59, 72, 74, 78, 96, 98, 99], 'holdings': [1, 29, 37, 74, 75, 79], 'founded': [1, 4, 6, 14, 15, 18, 20, 27, 28, 29, 32, 37, 39, 41, 54, 73, 74, 78, 79, 91, 92, 93, 96, 97, 98, 99], 'includes': [1, 2, 4, 5, 7, 10, 18, 20, 24, 25, 27, 30, 31, 36, 37, 38, 42, 43, 44, 50, 52, 58, 63, 65, 69, 71, 73, 74, 77, 79, 84, 88, 90, 91, 92, 93, 96, 97, 98], 'i.e': [1, 5, 10, 14, 20, 25, 28, 36, 38, 43, 44, 48, 51, 52, 53, 54, 58, 62, 64, 71, 99], 'potential': [1, 6, 7, 10, 14, 18, 20, 29, 36, 37, 38, 39, 42, 50, 52, 53, 56, 57, 59, 60, 69, 74, 75, 79, 82, 91, 93], 'involvement': [1, 15, 18, 20, 60